In [265]:
import pandas as pd
import numpy as np
from pathlib import Path
from create_reference_from_tsv_and_pepxml import ReferenceWriter
from create_PSM_df import PSM_FDR
from collections import defaultdict
from ReadAccTaxon import ReadAccTaxon
# load taxon graph
import sys  
sys.path.insert(0, '/home/jules/tax2proteome_projects/tax2proteome/')
from TaxonGraph import TaxonGraph
taxon_graph = TaxonGraph()
taxon_graph.create_graph("/home/jules/Documents/Metaproteomics/databases/databases_tax2proteome/taxdump.tar.gz")

path_to_kleiner_results_bachelor = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/"
path_to_tanca_results_bachelor = "/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_searchgui_xtandem_analyzer_bachelor_thesis/9MM_FASP/x_tandem_tsv"

uniprot_nr_reduced_tsv = {
    'subspecies': path_to_kleiner_results_bachelor + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_subspecies.t.xml_new_reduced.tsv",
    'species': path_to_kleiner_results_bachelor + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_species_nr.t.xml_new_reduced.tsv",
    'genus': path_to_kleiner_results_bachelor + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_genus_nr.t.xml_new_reduced.tsv",
    'family': path_to_kleiner_results_bachelor + "/uniprot_kleiner/x_tandem_tsv/Run1_U1_2000ng_uniprot_family_nr.t.xml_new_reduced.tsv",
}

tanca_uniprot_tsv = {
    'species': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_species_nr.t.xml_new_reduced.tsv",
    'genus': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_genus.t.xml_new_reduced.tsv",
    'family': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_family_nr.t.xml_new_reduced.tsv",
    'order': path_to_tanca_results_bachelor + "/9MM_FASP_uniprot_Tanca_order_nr.t.xml_new_reduced.tsv"
}
#species_name, species_taxon, %protein
# "name", taxa list species, taxa list level %protein
Kleiner_species_taxa = [
    ("Escherichia coli", [562], [562], 5.788), 
    ("Salmonella enterica", [28901], [28901], 33.773), 
    ("Bacillus subtilis", [1423], [1423], 0.788), 
    ("Staphylococcus aureus", [1280], [1280], 2.605), 
    ("Desulfovibrio vulgaris", [881], [881], 0.946), 
    ("Thermus thermophilus", [274], [274], 1.68),  
    ("Chlamydomonas reinhardtii", [3055], [3055], 3.996), 
    ("Paracoccus denitrificans", [266], [266], 0.922), 
    ("Nitrososphaera viennensis", [1034015], [1034015], 0.819), 
    ("Stenotrophomonas maltophilia", [40324], [40324], 8.021), 
    ("Altermonas macleodii", [28108], [28108], 0.954), 
    ("Chromobacterium violaceum", [536], [536], 1.259), 
    ("Paraburkholderia xenovorans", [36873], [36873], 0.433), 
    ("Cupriavidus metallidurans", [119219], [119219], 15.519),
    ("Nitrosomonas ureae", [44577], [44577], 0.543), 
    ("Nitrosomonas europaea", [915], [915], 0.082), 
    ("Nitrosospira multiformis", [1231], [1231], 0.209),
    ("Agrobacterium fabrum", [1176649], [1176649], 5.647), 
    ("Rhizobium leguminosarum", [384], [384], 3.172), 
    ("Pseudomonas fluorescens", [294], [294], 6.696), 
    ("Pseudomonas furukawaii", [1149133], [1149133], 1.165), 
    ("Pseudomonas sp. ATCC 13867", [1294143], [1294143], 2.871) 
]
#genus_name species_taxa, genus_taxa, %protein
Kleiner_genus_groups = [ 
    ("Nitrosomonas", [44577, 915],  [914], 0.644), 
    ("Pseudomonas", [294, 1149133, 1294143], [286], 10.733),
    ("viruses", [10754, 101570, 1985310, 329852, 1977402], [186794, 196894, 1198140, 11990, 10861], 0.513)
]

#family_name species_taxa, family_taxa, %protein
Kleiner_family_groups = [
    ("Enterobacteriaceae", [28901, 562],  [543], 39.561), 
    ("Burkholderiaceae", [36873, 119219], [119060], 15.519),
    ("Nitrosomonadaceae", [44577, 915, 1231],  [206379], 0.834), 
    ("Rhizobiaceae", [1176649, 384],  [82115], 8.819), 
    ("Pseudomonadaceae", [294, 1149133, 1294143], [135621], 10.733),
    ("viruses", [10754, 101570, 1985310, 329852, 1977402], [10744, 10699, 10662, 11989, 10860], 0.513)
]

Kleiner_groups = {'virus': [10754, 101570, 1985310, 329852, 1977402]}

Kleiner_level_groups = {'genus': [[44577, 915], [294, 1149133, 1294143]],
                        'family': [[28901, 562], [36873, 119219],  [44577, 915, 1231],  [1176649, 384], 
                           [294, 1149133, 1294143]]
                       }

Tanca_species_taxa = [
    ("Pasteurella multocida", [747], 11.1), ("Brevibacillus laterosporus", [1465], 11.1), 
    ("Lactobacillus acidophilus", [1579], 11.1), ("Lactobacillus casei", [1582], 11.1),
    ("Pediococcus pentosaceus", [1255], 11.1), ("Enterococcus faecalis", [1351], 11.1), 
    ("Rhodotorula glutinis", [5535], 11.1), ("Saccharomyces cerevisiae", [4932], 11.1),
    ("Escherichia coli", [562], 11.1)
]
Tanca_level_groups = {'genus': [[1579,1582]], 'family': [[1579,1582,1255]]}

In [13]:
# functions for loading xtandem results into df
def get_hit_rows2(decoy_column):
    return [True if d_set in  [{True, False}, {False}] else False for d_set in decoy_column]

def get_psm_and_df_in_fdr(file, fdr, remove_one_charged_spectra=False, columns=None):
    cs = ['Protein', 'Hyperscore', 'decoy', 'taxID']
    if columns:
        cs = cs + columns
    reduced_df = ReferenceWriter.read_csv_with_generic_function(file, cs, remove_one_charged_spectra)
    fdr_pos_result, number_psm_result, number_decoy_result, double_spectra_result, score_last_item_result = PSM_FDR.determine_FDR_position(reduced_df, fdr)
    return number_psm_result, reduced_df[0:fdr_pos_result]

def get_df_in_fdr_without_decoy(file, fdr, remove_one_charged_spectra=True, columns=None):
    df = get_psm_and_df_in_fdr(file, fdr, remove_one_charged_spectra, columns)[1]
    df = df[get_hit_rows2(df.decoy)]
    return df

def load_dfs(tsv_dict, level_list=None):
    fdr=0.05
    if not level_list:
        level_list = ['species', 'genus', 'family']
    result_dfs= {}
    for level in level_list:
        tsv = tsv_dict[level]
        df_in_fdr = get_df_in_fdr_without_decoy(tsv, fdr, columns=[f'taxID_{level}'])
        df_in_fdr.Protein = df_in_fdr.Protein.apply(lambda acc_set: {acc.split('|')[1] for acc in acc_set})
        result_dfs[level] = df_in_fdr    
    return result_dfs['species'], result_dfs['genus'], result_dfs['family']

def get_acc2taxid_dict(all_accs):
    final_accs = set()
    for acc in all_accs:
        try:
            final_accs.add(acc.split('|')[1])
        except:
            final_accs.add(acc)
    acc2tax_reader=ReadAccTaxon("/home/jules/Documents/Metaproteomics/databases/databases_tax2proteome/", "uniprot")
    acc_to_taxid_dict = acc2tax_reader.read_acc2tax(final_accs)
    acc_to_taxid_dict = {key: int(taxid) for key, taxid in acc_to_taxid_dict.items()}
    return acc_to_taxid_dict


In [125]:
def get_all_exclusive_rows(level_taxa_column):
    l=[]
    for t_set in level_taxa_column:
        if type(t_set) == list:
            t_set=set(t_set)
        if 'CRAP' in t_set:
            t_set.remove('CRAP')
        if 'DECOY' in t_set:
            t_set.remove('DECOY')
        if len(t_set)== 1:
            l.append(True)
        else:
            l.append(False)
    return l

def get_all_non_exclusive_rows(level_taxa_column):
    l=[]
    for t_set in level_taxa_column:
        if type(t_set) == list:
            t_set=set(t_set)
        if 'CRAP' in t_set:
            t_set.remove('CRAP')
        if 'DECOY' in t_set:
            t_set.remove('DECOY')
        if len(t_set) > 1:
            l.append(True)
        else:
            l.append(False)
    return l

def get_taxa_rows(column, taxID):
    if type(taxID)==int:
        return [True if taxID in t_set else False for t_set in column]
    elif type(taxID)==list:
        return [True if len(set(taxID).intersection(set(taxa_set)))>0 else False for taxa_set in column]

In [24]:
#um PSMs mit weniger als threshold nb PSMs zu entfernen muss acca zu taxid passende listen erstellen
def get_taxid_of_acc(acc, level, taxon_graph, acc_to_taxid_dict):
    if 'REVERSED' in acc:
        return "DECOY"
    try:
        taxid = taxon_graph.find_level_up(acc_to_taxid_dict[acc], level)
    except KeyError:
        taxid = "CRAP"
    return taxid

def get_taxids_of_accs(acc_list, level, taxon_graph, acc_to_taxid_dict):
    taxid_list = []
    for acc in acc_list:
        taxid_list.append(get_taxid_of_acc(acc, level, taxon_graph, acc_to_taxid_dict))
    return taxid_list

def sort_taxid_and_acc_in_df(df, level, acc_to_taxid_dict):
    df.Protein = df.Protein.apply(lambda acc_set: sorted(list(acc_set)))
    df[f"taxID_{level}"] = df.Protein.apply(lambda acc_list: get_taxids_of_accs(acc_list, level, taxon_graph, acc_to_taxid_dict))
    return df


In [5]:
df_in_fdr_uniprot_species, df_in_fdr_uniprot_genus, df_in_fdr_uniprot_family = load_dfs(uniprot_nr_reduced_tsv)

In [14]:
uniprot_accs_set = {item for sublist in df_in_fdr_uniprot_species.Protein for item in sublist}
uniprot_accs_set = uniprot_accs_set.union({item for sublist in df_in_fdr_uniprot_genus.Protein for item in sublist})
uniprot_accs_set = uniprot_accs_set.union({item for sublist in df_in_fdr_uniprot_family.Protein for item in sublist})
acc_to_taxid_dict = get_acc2taxid_dict(uniprot_accs_set)

Start reading accession2prot database file with 8 threads.
10% read.
20% read.
30% read.
40% read.
50% read.
60% read.
70% read.
80% read.
90% read.


In [25]:
df_in_fdr_uniprot_species_sorted = sort_taxid_and_acc_in_df(df_in_fdr_uniprot_species.copy(deep=True), 'species', acc_to_taxid_dict)
df_in_fdr_uniprot_genus_sorted = sort_taxid_and_acc_in_df(df_in_fdr_uniprot_genus.copy(deep=True), 'genus', acc_to_taxid_dict)
df_in_fdr_uniprot_family_sorted = sort_taxid_and_acc_in_df(df_in_fdr_uniprot_family.copy(deep=True), 'family', acc_to_taxid_dict)

In [30]:
# gibt Überschneidungen zwischen unique und multiple identifizierten spectra, wenn verschiedene 
# Peptide durch dasselbe spectrum identifiziert wurden, Summe ist also etwas höher
uniprot_genus_exclusive = df_in_fdr_uniprot_genus[get_all_exclusive_rows(df_in_fdr_uniprot_genus[f"taxID_genus"])]
uniprot_genus_non_exclusive = df_in_fdr_uniprot_genus[get_all_non_exclusive_rows(df_in_fdr_uniprot_genus[f"taxID_genus"])]
print("number all Spectra genus: ",len(set(df_in_fdr_uniprot_genus.Title)))
print("number exclusive Spectra genus: ", len(set(uniprot_genus_exclusive.Title)))
print("number non-exclusive Spectra genus: ",len(set(uniprot_genus_non_exclusive.Title)))
print("sum: ",len(set(uniprot_genus_exclusive.Title)) + len(set(uniprot_genus_non_exclusive.Title)))

print("all taxon 379 Spectra genus unsorted: ",len(set(df_in_fdr_uniprot_genus[get_taxa_rows(df_in_fdr_uniprot_genus[f"taxID_genus"], 379)].Title)))
set_exclusive = (set(uniprot_genus_exclusive[get_taxa_rows(uniprot_genus_exclusive[f"taxID_genus"], 379)].Title))
set_unique = (set(uniprot_genus_non_exclusive[get_taxa_rows(uniprot_genus_non_exclusive[f"taxID_genus"], 379)].Title))
set_intersection = (set_exclusive.intersection(set_unique))
print(len(set_exclusive), len(set_unique), len(set_exclusive)+len(set_unique)-len(set_intersection))

# sorted --> beide Zahlen gleich, kein Fehler mehr
uniprot_genus_exclusive_sorted = df_in_fdr_uniprot_genus_sorted[get_all_exclusive_rows(df_in_fdr_uniprot_genus_sorted[f"taxID_genus"])]
uniprot_genus_non_exclusive_sorted = df_in_fdr_uniprot_genus_sorted[get_all_non_exclusive_rows(df_in_fdr_uniprot_genus_sorted[f"taxID_genus"])]
print("sorted spectra")
print("number all Spectra genus: ",len(set(df_in_fdr_uniprot_genus_sorted.Title)))
print("number exclusive Spectra genus: ", len(set(uniprot_genus_exclusive_sorted.Title)))
print("number non-exclusive Spectra genus: ",len(set(uniprot_genus_non_exclusive_sorted.Title)))
print(len(set(df_in_fdr_uniprot_genus_sorted[get_taxa_rows(df_in_fdr_uniprot_genus_sorted[f"taxID_genus"], 379)].Title)))
print("sorted spectra taxon 379")
set_exclusive_sorted = (set(uniprot_genus_exclusive_sorted[get_taxa_rows(uniprot_genus_exclusive_sorted[f"taxID_genus"], 379)].Title))
set_unique_sorted = (set(uniprot_genus_non_exclusive_sorted[get_taxa_rows(uniprot_genus_non_exclusive_sorted[f"taxID_genus"], 379)].Title))
set_intersection_sorted = (set_exclusive_sorted.intersection(set_unique_sorted))
print(len(set_exclusive_sorted), len(set_unique_sorted), len(set_exclusive_sorted)+len(set_unique_sorted)-len(set_intersection_sorted))

print(set_unique.difference(set_unique_sorted))


number all Spectra genus:  35639
number exclusive Spectra genus:  21755
number non-exclusive Spectra genus:  15653
sum:  37408
all taxon 379 Spectra genus unsorted:  3619
1092 2691 3619
sorted spectra
number all Spectra genus:  35639
number exclusive Spectra genus:  21755
number non-exclusive Spectra genus:  15653
3619
sorted spectra taxon 379
1092 2691 3619
set()


In [ ]:
print(df_in_fdr_uniprot_genus[df_in_fdr_uniprot_genus.Title =='Run1_U1_2000ng.60899.60899.2'])
print(df_in_fdr_uniprot_genus_sorted[df_in_fdr_uniprot_genus_sorted.Title =='Run1_U1_2000ng.60899.60899.2'])

In [267]:
# functions for reducing df to PSMs wit at least x numbers of matching PSMs
#nb_peptides 1 - 4 
def remove_acc_row(column, acc_to_remove_set):
    return [False if len(set(accs).difference(acc_to_remove_set))==0 else True for accs in column]

def remove_empty_rows(column):
    return [False if len(accs)==0 else True for accs in column]

def add_tax_information(acc_list, acc2tax_dict, level):
    tax_list=[]
    for acc in acc_list:
        if 'REVERSED' in acc:
            tax_list.append('DECOY')
        else:
            try:
                tax_list.append(taxon_graph.find_level_up(acc2tax_dict[acc], level))
            except KeyError:
                tax_list.append('CRAP')
    return tax_list

In [269]:
def get_taxon_rows(column, taxon):
    return [True if taxon in t_list else False for t_list in column]

def count_all_identified_spectra_per_taxa(df, level, taxa_list):
    taxon_to_spectra_count_dict={}
    for taxon in taxa_list:
        taxon_to_spectra_count_dict[taxon] = len(set(df[get_taxon_rows(df[f"taxID_{level}"], taxon)].Title))
    return taxon_to_spectra_count_dict

def remove_acc_row(column, acc_to_remove_set):
    return [False if len(set(accs).difference(acc_to_remove_set))==0 else True for accs in column]

def remove_empty_rows(column):
    return [False if len(accs)==0 else True for accs in column]

def remove_all_accs_with_less_then_x_peptides(df, level, nb_peptides, acc2tax_dict):
    acc_to_peptide_dict=defaultdict(set)
    accs_to_remove= set()
    accs_to_keep = set()
    for index, row in df.iterrows():
        for acc in row['Protein']:
            if acc!="DECOY" and acc!="CRAP":
                acc_to_peptide_dict[acc].add(row["Peptide"])
    for acc, pep_set in acc_to_peptide_dict.items():
        if len(pep_set) < nb_peptides:
            accs_to_remove.add(acc)
        else:
            accs_to_keep.add(acc)
    df_no_x_hits=df[remove_acc_row(df.Protein, accs_to_remove)]
    df_no_x_hits.Protein = df_no_x_hits.Protein.apply(lambda acc_list: set(acc_list).intersection(accs_to_keep))
    df_no_x_hits.Protein = df_no_x_hits.Protein.apply(lambda acc_set: sorted(list(acc_set)))
    df_no_x_hits = df_no_x_hits[remove_empty_rows(df_no_x_hits.Protein)]
    df_no_x_hits[f"taxID_{level}"] = df_no_x_hits.Protein.apply(lambda acc_list: add_tax_information(acc_list, acc2tax_dict, level))
    return df_no_x_hits

def write_header(dataset, levels):
    fields = []
    for level in levels:
        fields.append(f"name {level}")
    for level in levels:
        fields.append(f"spectra count {level}")
        if level == "species":
            fields.extend([f"spectra count {level}"]*(len(levels)-1))
    for level in levels:
        fields.append(f"unique spectra count {level}")

    with open(f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_analysis/{dataset}_all_spectra_more_than_x_hits.tsv", "w") as out:
        out.write("\t".join(fields)) 
        out.write("\n")                   
    
    
def write_results(all_spectra_dict, unique_spectra_dict, species_taxa_list, level_groups, groups, dataset, levels):
    """
    param all_spectra_dict: {min_nb_psms:{taxon_str:{level:spectra_set}}}, spectra set includes all spectra identified for taxon
    param unique_spectra_dict: {min_nb_psms:{taxon_str:{level:spectra_set}}}, spectra set includes only spectra unique identified for taxon
        """
    with open(f"/home/jules/Documents/Metaproteomics/Tax2Proteome/benchmarking/results_analysis/{dataset}_all_spectra_more_than_x_hits.tsv", "a") as out:
        for min_nb_of_psms in all_spectra_dict.keys():
            out.write(f"Minimum number of PSMS per Protein to be counted: {min_nb_of_psms}\n")
            # row: name species, name genus, name family, spectra count species, ..., unique spectra count species, ...
            for species_taxon in species_taxa_list:
                row = [taxon_graph.get_scientific_name(species_taxon)]
                # names --> corect
                for level in levels[1:]:
                    level_taxon = taxon_graph.find_level_up(species_taxon, level)
                    row.append(taxon_graph.get_scientific_name(level_taxon))
                #spectral count   
                for level in levels:
                    row.append(str(len(all_spectra_dict[min_nb_of_psms][species_taxon][level])))
                    #add spectra count multiple related species for interesting level together
                    if level == "species":                        
                        for lev in levels[1:]:   
                            if any(species_taxon in x for x in level_groups[lev]):
                                taxa_list = [tax_list for tax_list in level_groups[lev] if species_taxon in tax_list][0]
                                spectra_set = set()
                                for tax in taxa_list:
                                    spectra_set = spectra_set.union(all_spectra_dict[min_nb_of_psms][tax]['species'])
                                row.append(str(len(spectra_set)))
                            else:
                                row.append("")
                #unique spectral count
                for level in levels: 
                    row.append(str(len(unique_spectra_dict[min_nb_of_psms][species_taxon][level])))
                out.write("\t".join(row)) 
                out.write("\n")
            # taxa groups
            if groups:
                for name, taxa_list in groups.items():
                    row = [name]*len(levels)
                    for level in levels:
                        row.append(str(len(all_spectra_dict[min_nb_of_psms][name][level])))
                        #add spectra count multiple related species for interesting level together
                        if level == "species":                        
                            for lev in levels[1:]:   
                                    row.append("")
                    for level in levels: 
                        row.append(str(len(unique_spectra_dict[min_nb_of_psms][name][level])))
                    out.write("\t".join(row)) 
                    out.write("\n")
            #last line
            out.write(f"all spectra\t")
            for i in range(len(levels)-1):
                        out.write("\t")
            for level in levels:
                out.write(f"{all_spectra_dict[min_nb_of_psms]['all'][level]}\t")
                if level == 'species':
                    for i in range(len(levels)-1):
                        out.write("\t")
            out.write("\n\n")
            

    
def extract_infomation_from_df(df_list, species_taxa_list, acc_to_taxid_dict, levels, groups):
    """
    param group: {name:[taxa list]}
    return all_spectra_dict: {min_nb_psms:{taxon_str:{level:spectra_set}}}, spectra set includes all spectra identified for taxon
    e.g. {2: {'562': {'species': {'Run1_U1_2000ng.41650.41650.2','Run1_U1_2000ng.76581.76581.3',...)
    return unique_spectra_dict: {min_nb_psms:{taxon_str:{level:spectra_set}}}, spectra set includes only spectra unique identified for taxon
    """
    # df_list have to be in correct order species, genus ...
    # min_nb_of_psms : {level: {taxon: spectra_count}}
    # level_to_taxa_tuple_list_dict : {level: (name, taxa list species, taxa list level, % protein)}
    all_spectra_dict = {}
    unique_spectra_dict = {}
    for min_nb_of_psms in [1,2,3,4]:
        print(f"Processing results for minimum {min_nb_of_psms} PSMS per Protein to be counted.")
        all_spectra_dict[min_nb_of_psms] = defaultdict(dict)
        unique_spectra_dict[min_nb_of_psms] = defaultdict(dict)
        #create df with all accs removed with less than min_nb_of_psms spectra hits per protein
        for n, level in enumerate(levels):
            df_no_x_hits=remove_all_accs_with_less_then_x_peptides(df_list[n].copy(deep=True),
                                                                    level, min_nb_of_psms, acc_to_taxid_dict)
            df_no_x_hits_unique=df_no_x_hits[get_all_exclusive_rows(df_no_x_hits[f"taxID_{level}"])]
            print(f"dfs cleaned from unique hits for level {level}")
            for taxon in species_taxa_list:
                # all species taxa, determine genus ans family taxa
                taxon_level = taxon_graph.find_level_up(taxon, level)
                all_spectra_dict[min_nb_of_psms][taxon][level] = \
                 set(df_no_x_hits[get_taxa_rows(df_no_x_hits[f"taxID_{level}"], taxon_level)].Title)
                unique_spectra_dict[min_nb_of_psms][taxon][level] = \
                 set(df_no_x_hits_unique[get_taxa_rows(df_no_x_hits_unique[f"taxID_{level}"], taxon_level)].Title) 
                                               
            # virus all taxa counted together 
            if groups:
                for group_name, taxa_group in groups.items():
                    taxa_level = [taxon_graph.find_level_up(taxon, level) for taxon in taxa_group]
                all_spectra_dict[min_nb_of_psms][group_name][level] = \
                 set(df_no_x_hits[get_taxa_rows(df_no_x_hits[f"taxID_{level}"], taxa_level)].Title)
                unique_spectra_dict[min_nb_of_psms][group_name][level] = \
                 set(df_no_x_hits_unique[get_taxa_rows(df_no_x_hits_unique[f"taxID_{level}"], taxa_level)].Title)
            # all spectra
            all_spectra_dict[min_nb_of_psms]['all'][level] = len(set(df_no_x_hits.Title))
    return all_spectra_dict, unique_spectra_dict
    
def extract_information_from_df_and_write(df_list, species_taxa_list, acc_to_taxid_dict, levels=None, level_groups=None, groups=None, dataset="Kleiner"):
    if levels==None:
        levels=["species", "genus", "family"]
    all_spectra_dict, unique_spectra_dict = extract_infomation_from_df(df_list, species_taxa_list, 
                                                                       acc_to_taxid_dict, levels, groups)
    write_header(dataset, levels)
    write_results(all_spectra_dict, unique_spectra_dict, species_taxa_list, level_groups, groups, dataset, levels)
        

In [271]:
#Kleiner set, uniprot
levels=["species", "genus", "family"]
species_taxa_list = [tax_tuple[1][0] for tax_tuple in Kleiner_species_taxa]
df_list = [df_in_fdr_uniprot_species_sorted, df_in_fdr_uniprot_genus_sorted, df_in_fdr_uniprot_family_sorted]
extract_information_from_df_and_write(df_list, species_taxa_list, acc_to_taxid_dict, levels, Kleiner_level_groups, Kleiner_groups)

Processing results for minimum 1 PSMS per Protein to be counted.
dfs cleaned from unique hits for level species
dfs cleaned from unique hits for level genus


No ancestor with level genus of taxID 101570 exists. TaxID 101570 of level species is returned.


dfs cleaned from unique hits for level family
Processing results for minimum 2 PSMS per Protein to be counted.
dfs cleaned from unique hits for level species
dfs cleaned from unique hits for level genus


No ancestor with level genus of taxID 101570 exists. TaxID 101570 of level species is returned.


dfs cleaned from unique hits for level family
Processing results for minimum 3 PSMS per Protein to be counted.
dfs cleaned from unique hits for level species
dfs cleaned from unique hits for level genus


No ancestor with level genus of taxID 101570 exists. TaxID 101570 of level species is returned.


dfs cleaned from unique hits for level family
Processing results for minimum 4 PSMS per Protein to be counted.
dfs cleaned from unique hits for level species
dfs cleaned from unique hits for level genus


No ancestor with level genus of taxID 101570 exists. TaxID 101570 of level species is returned.


dfs cleaned from unique hits for level family


In [276]:
#load Tanca dfs and accs
tanca_df_in_fdr_uniprot_species, tanca_df_in_fdr_uniprot_genus, tanca_df_in_fdr_uniprot_family = load_dfs(tanca_uniprot_tsv)
tanca_accs_set = {item for sublist in tanca_df_in_fdr_uniprot_species.Protein for item in sublist}
tanca_accs_set = tanca_accs_set.union({item for sublist in tanca_df_in_fdr_uniprot_genus.Protein for item in sublist})
tanca_accs_set = tanca_accs_set.union({item for sublist in tanca_df_in_fdr_uniprot_family.Protein for item in sublist})
acc_to_taxid_dict_tanca = get_acc2taxid_dict(tanca_accs_set)


Start reading accession2prot database file with 8 threads.
10% read.
20% read.
30% read.
40% read.
50% read.
60% read.
70% read.
80% read.
90% read.


In [277]:
tanca_df_in_fdr_uniprot_species = sort_taxid_and_acc_in_df(tanca_df_in_fdr_uniprot_species, 'species', acc_to_taxid_dict_tanca)
tanca_df_in_fdr_uniprot_genus = sort_taxid_and_acc_in_df(tanca_df_in_fdr_uniprot_genus, 'genus', acc_to_taxid_dict_tanca)
tanca_df_in_fdr_uniprot_family = sort_taxid_and_acc_in_df(tanca_df_in_fdr_uniprot_family, 'family', acc_to_taxid_dict_tanca)

In [280]:
# count Tanca spectra
levels=["species", "genus", "family"]
species_taxa_list_tanca = [tax_tuple[1][0] for tax_tuple in Tanca_species_taxa]
df_list_tanca = [tanca_df_in_fdr_uniprot_species, tanca_df_in_fdr_uniprot_genus, tanca_df_in_fdr_uniprot_family]
extract_information_from_df_and_write(df_list_tanca, species_taxa_list_tanca, acc_to_taxid_dict_tanca, levels, Tanca_level_groups, None, "Tanca")


Processing results for minimum 1 PSMS per Protein to be counted.
dfs cleaned from unique hits for level species
dfs cleaned from unique hits for level genus
dfs cleaned from unique hits for level family
Processing results for minimum 2 PSMS per Protein to be counted.
dfs cleaned from unique hits for level species
dfs cleaned from unique hits for level genus
dfs cleaned from unique hits for level family
Processing results for minimum 3 PSMS per Protein to be counted.
dfs cleaned from unique hits for level species
dfs cleaned from unique hits for level genus
dfs cleaned from unique hits for level family
Processing results for minimum 4 PSMS per Protein to be counted.
dfs cleaned from unique hits for level species
dfs cleaned from unique hits for level genus
dfs cleaned from unique hits for level family


In [279]:
species_taxa_list_tanca

[747, 1465, 1579, 1582, 1255, 1351, 5535, 4932, 562]

In [253]:
#seperated functions for testing
levels=["species", "genus", "family"]
species_taxa_list = [tax_tuple[1][0] for tax_tuple in Kleiner_species_taxa]
df_list = [tanca_df_in_fdr_uniprot_species, tanca_df_in_fdr_uniprot_genus, df_in_fdr_uniprot_family_sorted]

all_spectra_dict, unique_spectra_dict = extract_infomation_from_df(df_list, species_taxa_list, acc_to_taxid_dict,
                                                                   levels, Kleiner_groups, None, dataset="Tanca")       



Processing results for minimum 1 PSMS per Protein to be counted.
dfs cleaned from unique hits for level species
dfs cleaned from unique hits for level genus


No ancestor with level genus of taxID 101570 exists. TaxID 101570 of level species is returned.


dfs cleaned from unique hits for level family
Processing results for minimum 2 PSMS per Protein to be counted.
dfs cleaned from unique hits for level species
dfs cleaned from unique hits for level genus


No ancestor with level genus of taxID 101570 exists. TaxID 101570 of level species is returned.


dfs cleaned from unique hits for level family
Processing results for minimum 3 PSMS per Protein to be counted.
dfs cleaned from unique hits for level species
dfs cleaned from unique hits for level genus


No ancestor with level genus of taxID 101570 exists. TaxID 101570 of level species is returned.


dfs cleaned from unique hits for level family
Processing results for minimum 4 PSMS per Protein to be counted.
dfs cleaned from unique hits for level species
dfs cleaned from unique hits for level genus


No ancestor with level genus of taxID 101570 exists. TaxID 101570 of level species is returned.


dfs cleaned from unique hits for level family


In [260]:
write_header('Kleiner', levels)
write_results(all_spectra_dict, unique_spectra_dict, species_taxa_list, Kleiner_level_groups, Kleiner_groups, 'Kleiner', levels)